# Setup

required gspred and oauth

Go to Google Develpment console and create a new project. Go to apis and services and enable google sheets. Go to credentials and create a service account. Access level Viewer. Create and download .json key file. Save keyfile to your project folder. 

In [52]:
from oauth2client.service_account import ServiceAccountCredentials
import gspread
import pandas as pd

scope = [
    'https://www.googleapis.com/auth/spreadsheets'
]

GOOGLE_KEY_FILE = 'my_api_key.json'

credentials = ServiceAccountCredentials.from_json_keyfile_name(GOOGLE_KEY_FILE, scope)
gc = gspread.authorize(credentials)

workbook_key = '1yZv9w9zRKwrGTaR-YzmAqMefw4wMlaXocejdxZaTs6w' 
workbook = gc.open_by_key(workbook_key)
sheet = workbook.get_worksheet(0)
values = sheet.get_all_values()
c_data_0 = pd.DataFrame(values[1:],columns=values[0])

In [2]:
c_data_0.head()

,Province/State,Country/Region,Last Update,Confirmed,Deaths,Recovered
0,Hubei,Mainland China,1/31/2020 19:00,7153,249,169
1,Zhejiang,Mainland China,1/31/2020 19:00,537,,14
2,Guangdong,Mainland China,1/31/2020 19:00,436,,10
3,Henan,Mainland China,1/31/2020 19:00,352,2,3
4,Hunan,Mainland China,1/31/2020 19:00,332,,3


Connection works, test df is ok

# Trying out the dataset

In [32]:
pipe_df = pd.DataFrame()

In [13]:
n_sheets = len(workbook.worksheets())

In [31]:
n_sheets_new = len(workbook.worksheets())-10

In [33]:
for i in range(n_sheets_new):
    t_wb = workbook.get_worksheet(i)
    t_s = t_wb.get_all_values()
    pipe_df = pd.concat([pipe_df,pd.DataFrame(t_s[1:],columns=t_s[0])], axis = 0, ignore_index=True)

In [34]:
pipe_df.to_csv('output_union_001.csv')

* unioning all sheets does not work because different number of columns and mismatch col names.
* alternatively union sheets starting from '27 january' sheet. No errors.

In [29]:
for i in range(n_sheets):
    t_wb = workbook.get_worksheet(i)
    t_s = t_wb.get_all_values()
    t_nm = 'output_'+str(i)+'.csv'
    pd.DataFrame(t_s[1:],columns=t_s[0]).to_csv(t_nm)

AttributeError: 'NoneType' object has no attribute 'get_all_values'

Download all sheets as backup

In [3]:
workbook.worksheets()

[<Worksheet 'Jan31_7pm' id:1562962385>,
 <Worksheet 'Jan31_2pm' id:527706386>,
 <Worksheet 'Jan30_930pm' id:1207243992>,
 <Worksheet 'Jan30_11am' id:1320052203>,
 <Worksheet 'Jan29_9pm' id:536002735>,
 <Worksheet 'Jan29_230pm' id:91004260>,
 <Worksheet 'Jan29_130pm' id:1368739505>,
 <Worksheet 'Jan28_11pm' id:174402790>,
 <Worksheet 'Jan28_6pm' id:1954658351>,
 <Worksheet 'Jan28_1pm' id:1703730149>,
 <Worksheet 'Jan27_830pm' id:1979823633>,
 <Worksheet 'Jan27_7pm' id:2089257782>,
 <Worksheet 'Jan27_9am' id:62708662>,
 <Worksheet 'Jan26_11pm' id:32656279>,
 <Worksheet 'Jan26_11am' id:430379785>,
 <Worksheet 'Jan25_10pm' id:1501063148>,
 <Worksheet 'Jan25_12pm' id:1380010768>,
 <Worksheet 'Jan25_12am' id:76578435>,
 <Worksheet 'Jan24_12pm' id:1511763698>,
 <Worksheet 'Jan24_12am' id:1389232671>,
 <Worksheet 'Jan23_12pm' id:142449061>,
 <Worksheet 'Jan22_12pm' id:638231677>,
 <Worksheet 'Jan22_12am' id:2102449870>]

Check sheetnames

###########################################

# Cleaning and Unioning the whole dataset

Transform 'x_[...]' sheets into a format that can be unioned with newer sheets

* inspect unformated sheets and group edits

In [68]:
x_sheets_list = [
    'Jan26_11pm', #ok
    'Jan26_11am', #(-[''Suspected'])
    'Jan25_10pm', #(-[''Suspected'])
    'Jan25_12pm', #(-[''Suspected'])
    'Jan25_12am', #(-[''Suspected'])
    'Jan24_12pm', #(-[''Suspected'])
    'Jan24_12am', #(-[''Suspected'])
    'Jan23_12pm', #(-[''Suspected']) ('Demised' to 'Deaths')
    'Jan22_12pm', #(-[''Suspected'])
    'Jan22_12am' # (-[''Suspected']) (chg. 'Data last updated' to 'Last Update')
]

x_sheets_list_01 = [
    'Jan26_11am', #(-[''Suspected'])
    'Jan25_10pm', #(-[''Suspected'])
    'Jan25_12pm', #(-[''Suspected'])
    'Jan25_12am', #(-[''Suspected'])
    'Jan24_12pm', #(-[''Suspected'])
    'Jan24_12am', #(-[''Suspected'])
    'Jan22_12pm' #(-[''Suspected'])
]

* transform sheets
* union into newer sheets

In [72]:
#create empty dataframes to hold sheets
data_transf_df = pd.DataFrame(columns=['Province/State', 'Country/Region', 'Last Update', 'Confirmed','Deaths', 'Recovered'])
data_Okformat_df = pd.DataFrame()

#Select sheets with Ok format. Last 10 sheets are in non-standard format. I loop over the workbook with a range that is -10 shorter than the full len of the workbook.

n_sheets_new = len(workbook.worksheets())-10

#load sheets with ok format (starting from 27th january)
for i in range(n_sheets_new):
    t_wb = workbook.get_worksheet(i)
    t_s = t_wb.get_all_values()
    data_Okformat_df = pd.concat([data_Okformat_df,pd.DataFrame(t_s[1:],columns=t_s[0])], axis = 0, ignore_index=True)
    
#Load and transform bad sheets (22-26th january)

for i in x_sheets_list:
    if i in x_sheets_list_01:
        t_ws = workbook.worksheet(i)
        t_val = t_ws.get_all_values()
        t_df = pd.DataFrame(t_val[1:],columns=t_val[0])
        t_df = t_df.drop(columns=['Suspected'])
        #t_frames = [data_transf_df, t_df]
        data_transf_df = pd.concat([data_transf_df, t_df], axis = 0, ignore_index=True, sort=True)
    elif i == 'Jan23_12pm':
        t_ws = workbook.worksheet(i)
        t_val = t_ws.get_all_values()
        t_df = pd.DataFrame(t_val[1:],columns=t_val[0])
        t_df = t_df.drop(columns=['Suspected'])
        t_df = t_df.rename(columns={'Demised':'Deaths'})
        data_transf_df = pd.concat([data_transf_df, t_df], axis = 0, ignore_index=True, sort=True)
    elif i == 'Jan22_12am':
        t_ws = workbook.worksheet(i)
        t_val = t_ws.get_all_values()
        t_df = pd.DataFrame(t_val[1:],columns=t_val[0])
        t_df = t_df.drop(columns=['Suspected'])
        t_df = t_df.rename(columns={'Data last updated':'Last Update'})
        data_transf_df = pd.concat([data_transf_df, t_df], axis = 0, ignore_index=True, sort=True)
        
#Union two above DataFrames

complete_union_df = pd.concat([data_Okformat_df, data_transf_df], axis = 0, ignore_index=True, sort=True)

* check data

In [74]:
complete_union_df.describe()

,Confirmed,Country,Country/Region,Date last updated,Deaths,Last Update,Province/State,Recovered
count,1078,65,1013,27,1013,1051,1078,1013
unique,150,6,31,1,14,23,46,25
top,1,China,Mainland China,1/21/2020,,1/25/2020 12:00 PM,,
freq,179,57,605,27,891,86,227,774


In [76]:
complete_union_df.head()

,Confirmed,Country,Country/Region,Date last updated,Deaths,Last Update,Province/State,Recovered
0,7153,NaN,Mainland China,NaN,249,1/31/2020 19:00,Hubei,169
1,537,NaN,Mainland China,NaN,,1/31/2020 19:00,Zhejiang,14
2,436,NaN,Mainland China,NaN,,1/31/2020 19:00,Guangdong,10
3,352,NaN,Mainland China,NaN,2,1/31/2020 19:00,Henan,3
4,332,NaN,Mainland China,NaN,,1/31/2020 19:00,Hunan,3


In [77]:
complete_union_df.tail()

,Confirmed,Country,Country/Region,Date last updated,Deaths,Last Update,Province/State,Recovered
1073,,China,NaN,1/21/2020,NaN,NaN,Heilongjiang,NaN
1074,1,Japan,NaN,1/21/2020,NaN,NaN,,NaN
1075,2,Thailand,NaN,1/21/2020,NaN,NaN,,NaN
1076,1,South Korea,NaN,1/21/2020,NaN,NaN,,NaN
1077,1,United States,NaN,1/21/2020,NaN,NaN,Washington,NaN


### Write output

In [78]:
complete_union_df.to_csv('complete_union.csv')